In [ ]:
import torch as t
from torch import Tensor, nn
from torch.nn.functional import normalize
import transformer_lens
from transformer_lens import HookedTransformer
from dataclasses import dataclass
import math
import torch.nn.functional as F
import wandb
from sparse_QK_trainer import SparseQK, train_sparse_QK

In [ ]:
model = transformer_lens.HookedTransformer.from_pretrained("gpt2", fold_ln = True)

In [ ]:
batch_size = 20

import datasets
dataset = datasets.load_dataset("NeelNanda/pile-10k", split="train")
print(dataset)
print(dataset[0]['text'][:100])
tokens_dataset = transformer_lens.utils.tokenize_and_concatenate(dataset, model.tokenizer, streaming=False, max_length=20, column_name="text", add_bos_token=True, num_proc=4)
data_loader = t.utils.data.DataLoader(tokens_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

In [ ]:
cfg = {"d_hidden": 4096,
       "l1_coeff": 1e-6,
       "d_model": 768,
       "n_heads": 1,
       "d_head": 64,
       "seed": 87,
       "device": "cuda:0",
       "dead_freq": 1e-7}

In [ ]:
sparse_model = train_sparse_QK(model, cfg, 1, 10, data_loader)

In [1]:
import torch

In [51]:
a = torch.rand(12000, 2024)
b = torch.rand(12000, 2024)

a[:, 0] = 3.5
a[:, -1] = 4
a[:, 200] = 3.5
a[:, 40] = 4

b[:, 0] = 3
b[:, 200] = 6
b[:, -1] = 3
b[:, 40] = 4

a = a.log_softmax(-1)
b = b.log_softmax(-1)

In [49]:
a[0]

tensor([-2.8162, -8.1875, -8.7133,  ..., -8.7480, -7.8456, -1.8162])

In [43]:
kl = torch.nn.KLDivLoss(reduction = 'batchmean', log_target = True)

In [52]:
kl(a, b)

tensor(0.2334)

In [45]:
torch.exp(a)

tensor([[2.4049e-01, 1.4139e-05, 1.2405e-05,  ..., 2.0183e-05, 1.1473e-05,
         2.4049e-01],
        [2.4041e-01, 2.6309e-05, 2.5727e-05,  ..., 1.3227e-05, 2.6517e-05,
         2.4041e-01],
        [2.4051e-01, 1.1379e-05, 1.6808e-05,  ..., 1.6009e-05, 1.4333e-05,
         2.4051e-01],
        ...,
        [2.4053e-01, 1.1605e-05, 1.4606e-05,  ..., 1.4004e-05, 2.8144e-05,
         2.4053e-01],
        [2.4060e-01, 1.5101e-05, 1.5010e-05,  ..., 2.7902e-05, 1.4501e-05,
         2.4060e-01],
        [2.4053e-01, 1.3397e-05, 1.6657e-05,  ..., 1.2777e-05, 1.9051e-05,
         2.4053e-01]])